In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd
import torch as t
import os 
import pickle as pkl
from scipy.spatial import distance_matrix
import scipy
from tqdm.notebook import tqdm

from rewards import Perceptron, MLP, GCN, GRAPHCONV, plot_reward_correlation, plot_reward_spatial_patterns
from causal_maxent import irl_causal,stochastic_value_iteration

import warnings
import osmnx as ox
import contextily as cx
import matplotlib.pylab as pylab
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib_map_utils.core.scale_bar import ScaleBar, scale_bar
from matplotlib_map_utils.core.north_arrow import NorthArrow, north_arrow
import seaborn as sns

import pyproj
import math
import geopandas as gpd
from shapely import Point, LineString, Polygon


from transitions import get_transition, plot_transition
from trajectories import plot_trajectories, plot_trajectories_stats, generate_one_traj

In [ ]:
%load_ext autoreload
%autoreload 2

# Load Data

## Load Utils
Extract indices of buildings of study areas

In [ ]:
# translation 
translation = pd.read_csv("data/uniq_id_translation_2.csv", delimiter=";") 

In [ ]:
# discount factor for trajectory generation
discount = pd.read_csv("data/discount_factor.csv", usecols=[2,3])

## Load buildings
Extract buildings of study areas

In [ ]:
buildings = pd.read_csv("data/jerusalem_buildings_USG_region.csv", usecols=[2,3,4,8,9,13],index_col=0).loc[translation.UNIQ_ID.to_numpy()]
buildings.floorspace = buildings.floorspace.fillna(buildings.floorspace.mean()) # fill in missing values with mean

# Transition Matrix
Choose either simple gravity model or inverse euclidean distance

In [ ]:
transition = get_transition(buildings=buildings, use_simple_gravity=True, density=0.1)

In [ ]:
plot_transition(buildings=buildings, transitions=[transition.clone()], model_types=["Simple Gravity"],ncols=1)

# Compute initial and terminal States

In [ ]:
p_initial = t.ones(len(buildings)) / len(buildings)
terminal = (t.tensor(buildings.USG_CODE.to_numpy()) < 3).nonzero().flatten()

# IRL

In [ ]:
'''Import generic feature matrix'''

fm = t.tensor(pd.read_csv("data/IRL_feature_matrix.csv", index_col=0).loc[translation.UNIQ_ID.to_numpy()].to_numpy()).float()
initial_reward = fm.sum(1)

## Linear

In [ ]:
device = "cpu"
seeds = range(1) 
perceptrons = [Perceptron(fm.shape[1], seed).to(device) for seed in seeds]
lr = 8e-2  
decay = 0
optims = [t.optim.AdamW(perceptron.parameters(),lr, weight_decay=decay) for perceptron in perceptrons]

linear_rewards = []
linear_losses = []
linear_maxeigvals = []
for perceptron, optim in zip(perceptrons, optims):
    linear_reward, linear_loss,linear_maxeigval = irl_causal(fm.to(device), perceptron,optim, transition.float().to(device).clone(), p_initial.to(device), terminal.to(device), n_epochs=300, device=device)
    linear_rewards.append(linear_reward)
    linear_losses.append(linear_loss)
    linear_maxeigvals.append(linear_maxeigval)

## MLP

In [ ]:
device = "cpu"
seeds = range(1)
mlps = [MLP(fm.shape[1], seed).to(device) for seed in seeds]
lr = 4.5e-4
decay = 0
optims = [t.optim.AdamW(mlp.parameters(),lr, weight_decay=decay) for mlp in mlps]

mlp_rewards = []
mlp_losses = []
mlp_maxeigvals = []

for mlp, optim in zip(mlps, optims):
    mlp_reward, mlp_loss,mlp_maxeigval = irl_causal(fm.to(device), mlp,optim, transition.float().to(device).clone(), p_initial.to(device), terminal.to(device), n_epochs=300, device=device)
    mlp_rewards.append(mlp_reward)
    mlp_losses.append(mlp_loss)
    mlp_maxeigvals.append(mlp_maxeigval)

## GCN

In [ ]:
device = "cpu"

seeds = range(1)
transition = transition.float().to(device)
gcns = [GCN(fm.shape[1], transition, seed).to(device) for seed in seeds]

lr=8e-3
decay=0
optims = [t.optim.AdamW(gcn.parameters(),lr, weight_decay=decay) for gcn in gcns]

gcn_rewards = []
gcn_losses = []
gcn_maxeigvals = []
for gcn, optim in zip(gcns, optims):
    gcn_reward, gcn_loss,gcn_maxeigval  = irl_causal(fm.to(device), gcn, optim,transition.clone(), p_initial.to(device), terminal.to(device), n_epochs=300, device="cpu")
    gcn_rewards.append(gcn_reward)
    gcn_losses.append(gcn_loss)
    gcn_maxeigvals.append(gcn_maxeigval)

## GraphConv

In [ ]:
device = "cpu"

seeds = range(1)
transition = transition
graphconvs = [GRAPHCONV(fm.shape[1], transition, seed).to(device) for seed in seeds]

lr=8e-3
decay=0
optims = [t.optim.AdamW(graphconv.parameters(),lr, weight_decay=decay) for graphconv in graphconvs] 

graphconv_rewards = []
graphconv_losses = []
graphconv_maxeigvals = []
for graphconv, optim in zip(graphconvs, optims):
    graphconv_reward, graphconv_loss, graphconv_maxeigval  = irl_causal(fm.to(device), graphconv, optim,transition.clone(), p_initial.to(device), terminal.to(device), n_epochs=300, device="cpu")
    graphconv_rewards.append(graphconv_reward)
    graphconv_losses.append(graphconv_loss)
    graphconv_maxeigvals.append(graphconv_maxeigval)

# Analyse Reward

In [ ]:
normalize = lambda x : (x - x.min()) / (x.max() - x.min())

In [ ]:
normalized_linear = np.array([normalize(reward) for reward in linear_rewards])
normalized_mlp = np.array([normalize(reward) for reward in mlp_rewards])
normalized_gcn = np.array([normalize(reward) for reward in gcn_rewards])
normalized_graphconv = np.array([normalize(reward) for reward in graphconv_rewards])

normalized_linear_std = normalized_linear.std(0)
normalized_linear = normalized_linear.mean(0)
normalized_mlp_std = normalized_mlp.std(0)
normalized_mlp = normalized_mlp.mean(0)
normalized_gcn_std = normalized_gcn.std(0)
normalized_gcn = normalized_gcn.mean(0)
normalized_graphconv_std = normalized_graphconv.std(0)
normalized_graphconv = normalized_graphconv.mean(0)

In [ ]:
buildings["initial_reward"] = (initial_reward - initial_reward.min()) / (initial_reward.max() - initial_reward.min())

buildings["linear_reward"] = normalized_linear
buildings["linear_reward_std"] = normalized_linear_std
buildings["linear_diff"] = (buildings["linear_reward"] - buildings["initial_reward"])

buildings["mlp_reward"] = normalized_mlp
buildings["mlp_reward_std"] = normalized_mlp_std
buildings["mlp_diff"] = (buildings["mlp_reward"] - buildings["initial_reward"])

buildings["gcn_reward"] = normalized_gcn
buildings["gcn_reward_std"] = normalized_gcn_std
buildings["gcn_diff"] = (buildings["gcn_reward"] - buildings["initial_reward"])

buildings["graphconv_reward"] = normalized_graphconv
buildings["graphconv_reward_std"] = normalized_graphconv_std
buildings["graphconv_diff"] = (buildings["graphconv_reward"] - buildings["initial_reward"])


### Rewards correlation

In [ ]:
buildings["connected"] = transition.bool().sum(0).numpy()
buildings["connected"] = normalize(buildings["connected"])
buildings.floorspace = normalize(buildings.floorspace)

# change buildings.floorspace to buildings["connected"] for inverse euclidean transition matrix
R_values = [scipy.stats.linregress(buildings.floorspace, buildings[item+"_reward"])[2] for item in ["mlp", "linear", "gcn", "graphconv"]]
plot_reward_correlation(buildings, buildings.floorspace, "Normalized Floorspace",R_values)

### Rewards Spatial Pattern

In [ ]:
plot_reward_spatial_patterns(buildings)

## Generate Trajectories

In [ ]:
'''
normal transition with normal reward
'''
policy_weight = 1
normalize = lambda x : policy_weight * (x - x.min()) / (x.max() - x.min())

linear_value = stochastic_value_iteration(transition, normalize(linear_rewards[0]), 0.1)
linear_policy = transition * normalize(linear_value)
linear_policy /= linear_policy.sum(1).reshape(-1,1)

mlp_value = stochastic_value_iteration(transition, normalize(mlp_rewards[0]), 0.1)
mlp_policy = transition * normalize(mlp_value)
mlp_policy /= mlp_policy.sum(1).reshape(-1,1)

gcn_value = stochastic_value_iteration(transition, normalize(gcn_rewards[0]), 0.1)
gcn_policy = transition * normalize(gcn_value)
gcn_policy /= gcn_policy.sum(1).reshape(-1,1)

graphconv_value = stochastic_value_iteration(transition, normalize(graphconv_rewards[0]), 0.1)
graphconv_policy = transition * normalize(graphconv_value)
graphconv_policy /= graphconv_policy.sum(1).reshape(-1,1)

In [ ]:
policy_weight = 1
normalize = lambda x : policy_weight * (x - x.min()) / (x.max() - x.min())

linear_policy = transition * buildings.linear_reward.to_numpy()
linear_policy /= linear_policy.sum(1).reshape(-1,1)

mlp_policy = transition * buildings.mlp_reward.to_numpy()
mlp_policy /= mlp_policy.sum(1).reshape(-1,1)

gcn_policy = transition * buildings.gcn_reward.to_numpy()
gcn_policy /= gcn_policy.sum(1).reshape(-1,1)

graphconv_policy = transition * buildings.graphconv_reward.to_numpy()
graphconv_policy /= graphconv_policy.sum(1).reshape(-1,1)

In [ ]:
"""
Change terminal state configuration in trajectories.py 
"""
n_traj = 6567
linear_traj = [generate_one_traj(linear_policy, terminal, discount.to_numpy(), buildings) for idx in tqdm(range(n_traj))]
mlp_traj = [generate_one_traj(mlp_policy, terminal, discount.to_numpy(), buildings)for idx in tqdm(range(n_traj))]
gcn_traj = [generate_one_traj(gcn_policy, terminal, discount.to_numpy(), buildings)for idx in tqdm(range(n_traj))]
graphconv_traj = [generate_one_traj(graphconv_policy, terminal, discount.to_numpy(), buildings)for idx in tqdm(range(n_traj))]

### Plot individual trajectories

In [ ]:
routines = [linear_traj, mlp_traj, gcn_traj, graphconv_traj]
agent_indices = [1585,2047,3240,4000]
plot_trajectories(buildings, routines, agent_indices, ncols=2)

### Plot trajectory stats

In [ ]:
plot_trajectories_stats((linear_traj, mlp_traj, gcn_traj, graphconv_traj), buildings, "Original")

# Hypothetical Scenarios

## Reduced Boundaries

In [ ]:
"""
Reduce study area
"""
gdf = gpd.GeoDataFrame(buildings,crs='EPSG:2039', geometry=gpd.points_from_xy(x=buildings.x, y=buildings.y)).to_crs(4326)
x, y = pyproj.Transformer.from_crs("EPSG:2039", 4326).transform(gdf.loc[:, "x"], gdf.loc[:, "y"])
gdf.loc[:, "x"] = y
gdf.loc[:, "y"] = x
boundaries = gdf.total_bounds

new_bounds = [35.209, 31.783, 35.218, 31.789]
new_x_bounds = [35.209, 35.218]
new_y_bounds = [31.783, 31.789]
gdf_reset_index = gdf.reset_index()
new_gdf = gdf_reset_index[(gdf_reset_index.x > new_x_bounds[0]) & (gdf_reset_index.x < new_x_bounds[1]) & (gdf_reset_index.y > new_y_bounds[0]) & (gdf_reset_index.y < new_y_bounds[1])]
excluded = gdf.within(new_gdf.set_index("UNIQ_ID"))

In [ ]:
"""
Plot reduced study area
"""
fig, axs = plt.subplots(1,1, dpi=300)
gdf["excluded"] = excluded
gdf["excluded"] = gdf["excluded"].map({ True: "Survived",False: "Excluded"})
colors = ["Grey", "Red"]
gdf.plot(column="excluded", markersize=10, ax=axs,cmap=mpl.colors.ListedColormap(colors),legend=True,legend_kwds={'loc': 'upper right',"fontsize":10, "markerscale":.5})
axs.set_yticks([])
axs.set_xticks([])
buildings_osmnx = ox.geometries.geometries_from_bbox(31.78122826, 31.78975691, 35.20582386, 35.21960772, tags = {'building':True} )
buildings_osmnx.plot(ax=axs,alpha=0.2, color="grey")
north_arrow(
    axs, scale=.2, shadow=False, location="upper left", rotation={"crs": new_gdf.crs, "reference": "center"}, label={"position": "bottom", "text": "North", "fontsize": 10})
scale_bar(axs, location="lower center", style="boxes", bar={"projection": new_gdf.crs, "major_div": 1, "max":200,
        "minor_div": 2,},labels={"loc": "below", "style": "major","fontsize":10},units={"loc": "bar", "fontsize": 10}, )

fig.tight_layout()
plt.show()

In [ ]:
'''
new transition with reduced study area
'''
new_transitions = transition[new_gdf.index.to_list()][:,new_gdf.index.to_list()].numpy()
new_transitions /= new_transitions.sum(1).reshape(-1,1)

policy_weight = 1
normalize = lambda x : policy_weight * (x - x.min()) / (x.max() - x.min())

linear_value = stochastic_value_iteration(new_transitions, normalize(new_gdf.linear_reward), 0.1).to_numpy()
linear_policy = t.tensor(new_transitions * normalize(linear_value))
linear_policy /= linear_policy.sum(1).reshape(-1,1)

mlp_value = stochastic_value_iteration(new_transitions, normalize(new_gdf.mlp_reward), 0.1).to_numpy()
mlp_policy = t.tensor(new_transitions * normalize(mlp_value))
mlp_policy /= mlp_policy.sum(1).reshape(-1,1)

gcn_value = stochastic_value_iteration(new_transitions, normalize(new_gdf.gcn_reward), 0.1).to_numpy()
gcn_policy = t.tensor(new_transitions * normalize(gcn_value))
gcn_policy /= gcn_policy.sum(1).reshape(-1,1)

graphconv_value = stochastic_value_iteration(new_transitions, normalize(new_gdf.graphconv_reward), 0.1).to_numpy()
graphconv_policy = t.tensor(new_transitions * normalize(graphconv_value))
graphconv_policy /= graphconv_policy.sum(1).reshape(-1,1)

In [ ]:
"""
Change terminal state configuration in trajectories.py 
"""
n_traj = 6567
buildings_ = buildings.loc[new_gdf.UNIQ_ID]
terminal_ = t.tensor((buildings_.USG_CODE < 3).to_numpy().flatten()).nonzero().flatten()

linear_traj = [generate_one_traj(linear_policy, terminal_, discount.to_numpy(), buildings_) for idx in tqdm(range(n_traj))]
mlp_traj = [generate_one_traj(mlp_policy, terminal_, discount.to_numpy(), buildings_)for idx in tqdm(range(n_traj))]
gcn_traj = [generate_one_traj(gcn_policy, terminal_, discount.to_numpy(), buildings_)for idx in tqdm(range(n_traj))]
graphconv_traj = [generate_one_traj(graphconv_policy, terminal_, discount.to_numpy(), buildings_)for idx in tqdm(range(n_traj))]

In [ ]:
plot_trajectories_stats((linear_traj, mlp_traj, gcn_traj, graphconv_traj), buildings_, "Reduced Boundaries")

## Random Removal

In [ ]:
"""
Earthquakes
"""
survived = np.random.choice(len(gdf),size=int(len(gdf) * 0.7), replace=False)
new_gdf = gdf.iloc[survived].reset_index()
excluded = gdf.within(new_gdf.set_index("UNIQ_ID"))

In [ ]:
"""
Plot reduced study area
"""
fig, axs = plt.subplots(1,1, dpi=300)
gdf["excluded"] = excluded
gdf["excluded"] = gdf["excluded"].map({ True: "Survived",False: "Excluded"})
colors = ["Grey", "Red"]
gdf.plot(column="excluded", markersize=10, ax=axs,cmap=mpl.colors.ListedColormap(colors),legend=True,legend_kwds={'loc': 'upper right',"fontsize":10, "markerscale":.5})
axs.set_yticks([])
axs.set_xticks([])
buildings_osmnx = ox.geometries.geometries_from_bbox(31.78122826, 31.78975691, 35.20582386, 35.21960772, tags = {'building':True} )
buildings_osmnx.plot(ax=axs,alpha=0.2, color="grey")
north_arrow(
    axs, scale=.2, shadow=False, location="upper left", rotation={"crs": new_gdf.crs, "reference": "center"}, label={"position": "bottom", "text": "North", "fontsize": 10})
scale_bar(axs, location="lower center", style="boxes", bar={"projection": new_gdf.crs, "major_div": 1, "max":200,
        "minor_div": 2,},labels={"loc": "below", "style": "major","fontsize":10},units={"loc": "bar", "fontsize": 10}, )

'''Remove ColorBar'''
fig.tight_layout()
plt.show()

In [ ]:
'''
new transition with reduced study area
'''
new_transitions = transition[new_gdf.index.to_list()][:,new_gdf.index.to_list()].numpy()
new_transitions /= new_transitions.sum(1).reshape(-1,1)

policy_weight = 1
normalize = lambda x : policy_weight * (x - x.min()) / (x.max() - x.min())

linear_value = stochastic_value_iteration(new_transitions, normalize(new_gdf.linear_reward), 0.1).to_numpy()
linear_policy = t.tensor(new_transitions * normalize(linear_value))
linear_policy /= linear_policy.sum(1).reshape(-1,1)

mlp_value = stochastic_value_iteration(new_transitions, normalize(new_gdf.mlp_reward), 0.1).to_numpy()
mlp_policy = t.tensor(new_transitions * normalize(mlp_value))
mlp_policy /= mlp_policy.sum(1).reshape(-1,1)

gcn_value = stochastic_value_iteration(new_transitions, normalize(new_gdf.gcn_reward), 0.1).to_numpy()
gcn_policy = t.tensor(new_transitions * normalize(gcn_value))
gcn_policy /= gcn_policy.sum(1).reshape(-1,1)

graphconv_value = stochastic_value_iteration(new_transitions, normalize(new_gdf.graphconv_reward), 0.1).to_numpy()
graphconv_policy = t.tensor(new_transitions * normalize(graphconv_value))
graphconv_policy /= graphconv_policy.sum(1).reshape(-1,1)

In [ ]:
"""
Change terminal state configuration in trajectories.py 
"""
n_traj = 6567
buildings_ = buildings.loc[new_gdf.UNIQ_ID]
terminal_ = t.tensor((buildings_.USG_CODE < 3).to_numpy().flatten()).nonzero().flatten()

linear_traj = [generate_one_traj(linear_policy, terminal_, discount.to_numpy(), buildings_) for idx in tqdm(range(n_traj))]
mlp_traj = [generate_one_traj(mlp_policy, terminal_, discount.to_numpy(), buildings_)for idx in tqdm(range(n_traj))]
gcn_traj = [generate_one_traj(gcn_policy, terminal_, discount.to_numpy(), buildings_)for idx in tqdm(range(n_traj))]
graphconv_traj = [generate_one_traj(graphconv_policy, terminal_, discount.to_numpy(), buildings_)for idx in tqdm(range(n_traj))]

In [ ]:
plot_trajectories_stats((linear_traj, mlp_traj, gcn_traj, graphconv_traj), buildings_, "Random Removal")